# Datasets

Incluye:

- Reducción de los datasets, para pruebas en local


In [ ]:
# Librerias
import os
import pandas as pd
import numpy as np
import datetime
import joblib

####################
# funciones y clases
from utils import psi, drift_deflacion

#######
# rutas
# datasets
from config import datasets_path, dataset_file_fe6_6xpqt
# optimizacion
from config import db_path
# modelos
from config import modelos_path
# predicciones
from config import pred_path

##########
# pipeline
from processing import ModelPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


import warnings

# Ignorar advertencias de tipo UserWarning
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')


Variables de train y test

In [ ]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202007, 202008, 202009, 202010, 202011, 202012, # sin 202006
                 202101, 202102, 202103, 202104, 202105, 202106, 202107]

mes_train_ult_3_meses = [202105, 202106, 202107]

mes_train_ult_6_meses = [202102, 202103, 202104, 202105, 202106, 202107]

mes_train_ult_9_meses = [202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107]

mes_train_ult_anio = [202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106, 202107]

mes_train = [202107]
mes_test = 202109

threshold = 0.025

semillas = [437809, 327347, 392879, 455783, 217163]

## Creating smaller datasets

Undersampling para optimización

10 % de los "CONTINUA"

In [ ]:
data = pd.read_parquet(dataset_file_fe6_6xpqt)

data.foto_mes.unique()

In [ ]:
# Set de train para optimizar
data = data[data['foto_mes'].isin(mes_train_all)]

# Filtrar las filas que tienen el valor "CONTINUA" en la columna clase_ternaria
df_continua = data[data['clase_ternaria'] == 'CONTINUA']

# Seleccionar el 10 % de las filas de "CONTINUA" (sin reemplazo)
df_continua_sample = df_continua.sample(frac=0.1, random_state=semillas[0])

del df_continua

# Filtrar las filas que no tienen el valor "CONTINUA" en la columna clase_ternaria
df_not_continua = data[data['clase_ternaria'] != 'CONTINUA']

del data

# Concatenar el sampleo de "CONTINUA" con el resto del dataframe
df_undersampled = pd.concat([df_continua_sample, df_not_continua])

df_undersampled.to_parquet(datasets_path+"competencia_02_fe6x_opt_under.parquet")